In [1]:
import os

import numpy as np
import torch
from lib.config import cfg

import lib.dataset.dataset3D as dataUtils
from lib.vortex.vortex import Vortex
from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Test_Crop')

#project.create_new(
#    name = 'Test_Crop',
#    dataset2D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen',
#    dataset3D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen')

cfg = project.get_cfg()

320
Successfully loaded project Test_Crop!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!

EfficientTrack Configuration:
Use suggested Bounding Box size of 320 pixels? (yes/no)
y
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!

Vortex Configuration:
Use suggested 3D Bounding Box size of 208 mm? (yes/no)
y
Use suggested grid spacing of 2 mm? (yes/no)
y
Use suggested x-Axis Grid Dimension of [-206, 172] mm? (yes/no)
y
Use suggested y-Axis Grid Dimension of [-270, 352] mm? (yes/no)
y
Use suggested z-Axis Grid Dimension of [316, 832] mm? (yes/no)
y


In [2]:
training_set = dataUtils.VortexDataset3D(cfg, set='train')
val_set = dataUtils.VortexDataset3D(cfg, set='val')
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [8]:
weights = '/home/timo/Desktop/VoRTEx/projects/Test/models/vortex/Run_20210705-185157/Vortex-d_30.pth'

efficienttrack_weights = '/home/timo/Desktop/VoRTEx/projects/Test/models/efficienttrack/EfficientTrack-d3_80_171153.pth'

vortexNet = Vortex('train', cfg, calibPaths, weights, None)

projects/Test_Crop/lookup.npy
[Info] loaded weights: Vortex-d_30.pth


In [5]:
vortexNet.train(training_set, val_set, 1000)

NameError: name 'vortexNet' is not defined